<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/3%D0%9A%D0%B5%D0%B9%D1%812_%D0%A1%D0%BA%D0%BE%D0%BB%D1%8C%D0%BA%D0%BE_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87_%D0%B2_%D1%81%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%BC_%D1%80%D0%B5%D1%88%D0%B0%D0%B5%D1%82%D1%81%D1%8F_%D0%B7%D0%B0_%D0%BE%D0%B4%D0%B8%D0%BD_%D0%B0%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D1%8B%D0%B9_%D1%81%D0%B5%D0%B0%D0%BD%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from datetime import datetime
from collections import defaultdict
import csv

def analyze(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader)
        rows = [row for row in reader]

    if not rows:
        print("Файл пуст")
        return 0.0

    rows.sort(key=lambda x: (x[1], x[0]))  # Сортировка строк по пользователю, по времени

    user_last_activity={} # Словарь пользователь-последняя дата активности
    # {'10056': datetime.datetime(2023, 4, 30, 22, 16, 4, 379039), '10094': datetime.datetime(2023, 4, 30, 20, 14, 35, 533798), ...}

    user_current_session_tasks={}  # Словарь пользователь - множество задач в текущем сеансе
    # {'10056': {'813'}, '10094': {'1111'}, '10185': {'737', '786', '785', '1041',....}

    all_sessions=[]  # Список пользователь - количество всех его сеансов
    # [('10214', 3), ('10250', 1), ('10593', 2), ('10668', 3), ('10673', 13), ...]

    for row in rows:
        date = row[0]
        user = row[1]
        prob = row[2]
        stat = row[3]

        try:
            time = datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
        except Exception as e:
            print(f"Некорректная дата: {e}")
            continue

        if user not in user_last_activity: # Подсчет активных сеансов для всех пользователей
            user_current_session_tasks[user]=set([prob])
            user_last_activity[user]=time
        else:
            time_diff=(time-user_last_activity[user]).total_seconds()

            if time_diff>3600: # Сохранение предыдущего сеанса
                if user in user_current_session_tasks: # Запись о пользователе-задаче есть
                    tasks_count=len(user_current_session_tasks[user]) # Количество текущих сеансов у пользователя
                    all_sessions.append((user, tasks_count)) # Добавление в список данных о пользователе и общем количестве сеансов

                user_current_session_tasks[user]=set([prob]) # Добавление данных о пользователе-задаче и времени - новый сеанс
            else: # Продолжение текущего сеанса
                user_current_session_tasks[user].add(prob)

            user_last_activity[user]=time

    # Сохраняем последние активные сеансы для каждого пользователя - для них нет следующего действия с разрывом >1ч
    for user in user_current_session_tasks:
        tasks_count=len(user_current_session_tasks[user])
        all_sessions.append((user, tasks_count))

    # Статистика по сеансам
    total_sessions=len(all_sessions) # Общее количество активных сеансов
    total_tasks_in_sessions=sum(tasks_count for _, tasks_count in all_sessions) # Общее количество задач во всех активных сеансах

    if total_sessions> 0: # Подсчет среднего количества решаемых задач за активный сеанс
        avg_tasks_per_session=total_tasks_in_sessions/total_sessions # Среднее количество задач на 1 активную сессию
    else:
        avg_tasks_per_session = 0.0

    print(f"{avg_tasks_per_session:.2f}")
    return round(avg_tasks_per_session,2)

res3=analyze('codesubmit.csv')


3.14
